In [2]:
import os
os.environ['JAVA_HOME']='/Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home'
print(os.environ['JAVA_HOME'])

/Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home


In [3]:
import pyspark 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
from pyspark.sql import DataFrameReader
from pyspark.sql import DataFrameWriter
from pyspark import SparkContext

In [4]:
#spark.stop

In [5]:
spark = SparkSession.builder.getOrCreate()
print(spark.version)

24/09/17 08:59:41 WARN Utils: Your hostname, Engins-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.116.46 instead (on interface en0)
24/09/17 08:59:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/17 08:59:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


3.5.1


In [6]:
# Reuse the existing SparkContext or create a new one if none exists
sc = SparkContext.getOrCreate()

In [7]:
#reading census_income data and remove white spaces in strings type columns from data:
import pyspark.sql.functions as func
census_path = '/Users/enginsan/Downloads/census_income.csv'

census = spark.read.csv(
    census_path
    , header=True
    , inferSchema=True
)

for col, typ in census.dtypes:
    if typ == 'string':
        census = census.withColumn(
            col
            , func.ltrim(func.rtrim(census[col]))
        )
census.count()

32561

In [8]:
census.show(10)

+---+----------------+------+---------+-------------+--------------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+-----+
|age|       workclass|fnlwgt|education|education-num|      marital-status|       occupation| relationship| race|   sex|capital-gain|capital-loss|hours-per-week|native-country|label|
+---+----------------+------+---------+-------------+--------------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+-----+
| 39|       State-gov| 77516|Bachelors|           13|       Never-married|     Adm-clerical|Not-in-family|White|  Male|        2174|           0|            40| United-States|<=50K|
| 50|Self-emp-not-inc| 83311|Bachelors|           13|  Married-civ-spouse|  Exec-managerial|      Husband|White|  Male|           0|           0|            13| United-States|<=50K|
| 38|         Private|215646|  HS-grad|            9|            Divorced|Handlers-cleaner

In [9]:
census.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- education-num: integer (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital-gain: integer (nullable = true)
 |-- capital-loss: integer (nullable = true)
 |-- hours-per-week: integer (nullable = true)
 |-- native-country: string (nullable = true)
 |-- label: string (nullable = true)



In [10]:
census.dtypes 
#first var is col label, second is the type

[('age', 'int'),
 ('workclass', 'string'),
 ('fnlwgt', 'int'),
 ('education', 'string'),
 ('education-num', 'int'),
 ('marital-status', 'string'),
 ('occupation', 'string'),
 ('relationship', 'string'),
 ('race', 'string'),
 ('sex', 'string'),
 ('capital-gain', 'int'),
 ('capital-loss', 'int'),
 ('hours-per-week', 'int'),
 ('native-country', 'string'),
 ('label', 'string')]

In [11]:
census.dtypes[:-1]

[('age', 'int'),
 ('workclass', 'string'),
 ('fnlwgt', 'int'),
 ('education', 'string'),
 ('education-num', 'int'),
 ('marital-status', 'string'),
 ('occupation', 'string'),
 ('relationship', 'string'),
 ('race', 'string'),
 ('sex', 'string'),
 ('capital-gain', 'int'),
 ('capital-loss', 'int'),
 ('hours-per-week', 'int'),
 ('native-country', 'string')]

In [12]:
cols_to_keep = census.dtypes

cols_to_keep = (
    ['label','age'
     ,'capital-gain'
     ,'capital-loss'
     ,'hours-per-week'
    ] + [
        e[0] for e in cols_to_keep[:-1] 
        if e[1] == 'string'
    ]
)

cols_to_keep

['label',
 'age',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'workclass',
 'education',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native-country']

In [13]:
#Get numeric and categorical columns
import pyspark.mllib.stat as st
import numpy as np

census_subset = census.select(cols_to_keep)

cols_num = [
    e[0] for e in census_subset.dtypes 
    if e[1] == 'int'
]
cols_cat = [
    e[0] for e in census_subset.dtypes[1:] 
    if e[1] == 'string'
]
cols_num, cols_cat

(['age', 'capital-gain', 'capital-loss', 'hours-per-week'],
 ['workclass',
  'education',
  'marital-status',
  'occupation',
  'relationship',
  'race',
  'sex',
  'native-country'])

In [14]:
rdd_num = (
    census_subset
    .select(cols_num)
    .rdd
    .map(lambda row: [e for e in row])
)
rdd_num.take(5)

[[39, 2174, 0, 40],
 [50, 0, 0, 13],
 [38, 0, 0, 40],
 [53, 0, 0, 40],
 [28, 0, 0, 40]]

In [15]:
stats_num = st.Statistics.colStats(rdd_num)
print(stats_num)


In [16]:
for col, min_, mean_, max_, var_ in zip(
      cols_num
    , stats_num.min()
    , stats_num.mean()
    , stats_num.max()
    , stats_num.variance()
):
    print('{0}: min->{1:.1f}, mean->{2:.1f}, max->{3:.1f}, stdev->{4:.1f}'
          .format(col, min_, mean_, max_, np.sqrt(var_)))

age: min->17.0, mean->38.6, max->90.0, stdev->13.6
capital-gain: min->0.0, mean->1077.6, max->99999.0, stdev->7385.3
capital-loss: min->0.0, mean->87.3, max->4356.0, stdev->403.0
hours-per-week: min->1.0, mean->40.4, max->99.0, stdev->12.3


In [17]:
#categorical data:
rdd_cat = (
    census_subset
    .select(cols_cat + ['label'])
    .rdd
    .map(lambda row: [e for e in row])
)

results_cat = {}

for i, col in enumerate(cols_cat + ['label']):
    results_cat[col] = (
        rdd_cat
        .groupBy(lambda row: row[i])
        .map(lambda el: (el[0], len(el[1])))
        .collect()
    )
#el[0] is value, len(el[1] is frequency ) )
for k in results_cat:
    print(
        k
        , sorted(
            results_cat[k]
            , key=lambda el: el[1]
            , reverse=True)
        , '\n')

workclass [('Private', 22696), ('Self-emp-not-inc', 2541), ('Local-gov', 2093), ('?', 1836), ('State-gov', 1298), ('Self-emp-inc', 1116), ('Federal-gov', 960), ('Without-pay', 14), ('Never-worked', 7)] 

education [('HS-grad', 10501), ('Some-college', 7291), ('Bachelors', 5355), ('Masters', 1723), ('Assoc-voc', 1382), ('11th', 1175), ('Assoc-acdm', 1067), ('10th', 933), ('7th-8th', 646), ('Prof-school', 576), ('9th', 514), ('12th', 433), ('Doctorate', 413), ('5th-6th', 333), ('1st-4th', 168), ('Preschool', 51)] 

marital-status [('Married-civ-spouse', 14976), ('Never-married', 10683), ('Divorced', 4443), ('Separated', 1025), ('Widowed', 993), ('Married-spouse-absent', 418), ('Married-AF-spouse', 23)] 

occupation [('Prof-specialty', 4140), ('Craft-repair', 4099), ('Exec-managerial', 4066), ('Adm-clerical', 3770), ('Sales', 3650), ('Other-service', 3295), ('Machine-op-inspct', 2002), ('?', 1843), ('Transport-moving', 1597), ('Handlers-cleaners', 1370), ('Farming-fishing', 994), ('Tech-s

In [18]:
correlations = st.Statistics.corr(rdd_num)
correlations

24/09/17 09:00:02 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


array([[ 1.        ,  0.0776745 ,  0.05777454,  0.06875571],
       [ 0.0776745 ,  1.        , -0.03161506,  0.07840862],
       [ 0.05777454, -0.03161506,  1.        ,  0.05425636],
       [ 0.06875571,  0.07840862,  0.05425636,  1.        ]])

In [19]:
for i, el_i in enumerate(abs(correlations) > 0.05):
    print(cols_num[i])
    
    for j, el_j in enumerate(el_i):
        if el_j and j != i:
            print(
                '    '
                , cols_num[j]
                , correlations[i][j]
            )
            
    print()

age
     capital-gain 0.077674498166008
     capital-loss 0.05777453947896792
     hours-per-week 0.0687557075095634

capital-gain
     age 0.077674498166008
     hours-per-week 0.078408615390123

capital-loss
     age 0.05777453947896792
     hours-per-week 0.05425636227262224

hours-per-week
     age 0.0687557075095634
     capital-gain 0.078408615390123
     capital-loss 0.05425636227262224



In [20]:
#Testing the data:

In [41]:
import pyspark.mllib.linalg as ln

census_occupation = (
    census
    .groupby('occupation')
    .pivot('label')
    .count()
)

census_occupation_coll = (
    census_occupation
    .rdd
    .map(lambda row: (row[1:]))
    .flatMap(lambda row: row)
    .collect()
)

len_row = len(census_occupation.collect())
dense_mat = ln.DenseMatrix(
    len_row
    , 2
    , census_occupation_coll
    , True
)

chi_sq = st.Statistics.chiSqTest(dense_mat)

print(chi_sq.pValue)
print(chi_sq.nullHypothesis)

0.0
the occurrence of the outcomes is statistically independent.


In [43]:
dense_mat.toArray()

array([[2.667e+03, 9.830e+02],
       [2.098e+03, 1.968e+03],
       [2.281e+03, 1.859e+03],
       [1.284e+03, 8.600e+01],
       [8.790e+02, 1.150e+02],
       [3.170e+03, 9.290e+02],
       [1.277e+03, 3.200e+02],
       [1.480e+02, 1.000e+00],
       [4.380e+02, 2.110e+02],
       [3.158e+03, 1.370e+02],
       [6.450e+02, 2.830e+02],
       [1.752e+03, 2.500e+02],
       [8.000e+00, 1.000e+00],
       [1.652e+03, 1.910e+02],
       [3.263e+03, 5.070e+02]])

In [45]:
len_ftrs = []

for col in cols_cat:
    (
        len_ftrs
        .append(
            (col
             , census
                 .select(col)
                 .distinct()
                 .count()
            )
        )
    )
    
len_ftrs = dict(len_ftrs)
len_ftrs

{'workclass': 9,
 'education': 16,
 'marital-status': 7,
 'occupation': 15,
 'relationship': 6,
 'race': 5,
 'sex': 2,
 'native-country': 42}

In [47]:
import pyspark.mllib.feature as feat

final_data = (
    census
    .select(cols_to_keep)
    .rdd
    .map(lambda row: [
        list(
            feat.HashingTF(int(len_ftrs[col] / 2.0))
            .transform(row[i])
            .toArray()
        ) if i >= 5
        else [row[i]] 
        for i, col in enumerate(cols_to_keep)]
    )
)

final_data.take(3)

[[['<=50K'],
  [39],
  [2174],
  [0],
  [40],
  [4.0, 2.0, 2.0, 1.0],
  [0.0, 3.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
  [5.0, 3.0, 5.0],
  [1.0, 0.0, 3.0, 0.0, 0.0, 4.0, 4.0],
  [4.0, 4.0, 5.0],
  [1.0, 4.0],
  [4.0],
  [0.0,
   0.0,
   6.0,
   0.0,
   1.0,
   1.0,
   0.0,
   0.0,
   0.0,
   1.0,
   0.0,
   0.0,
   1.0,
   0.0,
   0.0,
   0.0,
   1.0,
   0.0,
   1.0,
   1.0,
   0.0]],
 [['<=50K'],
  [50],
  [0],
  [0],
  [13],
  [4.0, 4.0, 5.0, 3.0],
  [0.0, 3.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
  [8.0, 6.0, 4.0],
  [1.0, 1.0, 5.0, 0.0, 0.0, 4.0, 4.0],
  [3.0, 2.0, 2.0],
  [1.0, 4.0],
  [4.0],
  [0.0,
   0.0,
   6.0,
   0.0,
   1.0,
   1.0,
   0.0,
   0.0,
   0.0,
   1.0,
   0.0,
   0.0,
   1.0,
   0.0,
   0.0,
   0.0,
   1.0,
   0.0,
   1.0,
   1.0,
   0.0]],
 [['<=50K'],
  [38],
  [0],
  [0],
  [40],
  [1.0, 3.0, 0.0, 3.0],
  [2.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0],
  [2.0, 3.0, 3.0],
  [2.0, 0.0, 6.0, 0.0, 2.0, 6.0, 1.0],
  [4.0, 4.0, 5.0],
  [1.0, 4.0],
  [4.0],
  [0.0,
   0.0,
   6.0,


In [49]:
def labelEncode(label):
    return [int(label[0] == '>50K')]

final_data = (
    final_data
    .map(lambda row: labelEncode(row[0]) 
         + [item 
            for sublist in row[1:] 
            for item in sublist]
        )
)

In [51]:

standardizer = feat.StandardScaler(True, True)
sModel = standardizer.fit(final_data.map(lambda row: row[1:]))
final_data_scaled = sModel.transform(final_data.map(lambda row: row[1:]))

final_data = (
    final_data
    .map(lambda row: row[0])
    .zipWithIndex()
    .map(lambda row: (row[1], row[0]))
    .join(
        final_data_scaled
        .zipWithIndex()
        .map(lambda row: (row[1], row[0]))
    )
    .map(lambda row: row[1])
)

final_data.take(3)

[(0,
  DenseVector([0.0307, 0.1485, -0.2167, -0.0354, 2.1254, -0.8191, 0.6378, -2.2258, -1.08, 1.1674, 0.639, 0.3252, -0.4647, 0.0487, -0.5787, 0.8307, -0.3164, -0.9455, 1.059, -0.4371, -0.4331, -0.7278, -0.2899, -1.1361, 0.9505, 1.3817, 0.9265, 1.1796, 1.4008, -0.2909, -0.1283, -0.7031, -0.1436, -0.1934, 0.3297, -0.0222, 0.3265, 0.3019, -0.0858, -0.1071, 0.0, 0.2601, -0.0543, -0.1384, 0.1693, -0.2117, 0.0, -0.0901, 0.051, -0.0242, 0.3187, 0.043, -0.1431])),
 (0,
  DenseVector([-0.0426, -0.1459, -0.2167, -0.0354, -0.4816, 0.2868, -0.5428, 0.4723, 1.2753, -1.0583, -0.9617, 0.3252, -0.4647, 0.0487, -0.5787, 0.8307, -1.5392, -0.9455, -1.6947, 0.2373, -0.4331, 0.3617, -0.2899, 1.6247, 2.3798, -0.5208, 0.9265, 1.1796, 1.4008, -0.2909, -0.1283, -0.7031, -0.1436, -0.1934, 0.3297, -0.0222, 0.3265, 0.3019, -0.0858, -0.1071, 0.0, 0.2601, -0.0543, -0.1384, 0.1693, -0.2117, 0.0, -0.0901, 0.051, -0.0242, 0.3187, 0.043, -0.1431])),
 (0,
  DenseVector([-0.7758, -0.1459, -0.2167, -0.0354, -0.4816, 0.2

In [53]:
sModel.mean, sModel.std

(DenseVector([38.5816, 1077.6488, 87.3038, 40.4375, 1.5543, 2.7407, 0.9196, 2.6499, 0.9171, 1.4265, 0.6008, 0.8597, 1.4332, 0.9619, 1.6429, 0.5917, 5.7763, 4.4069, 4.2309, 1.6481, 0.1579, 5.0041, 0.0775, 0.823, 2.6699, 1.8212, 3.2491, 2.8515, 3.0191, 1.353, 4.186, 4.6616, 0.0202, 0.0468, 5.5312, 0.0005, 0.9028, 0.9152, 0.0073, 0.0113, 0.0, 0.9233, 0.0038, 0.0188, 0.9453, 0.0486, 0.0, 0.008, 0.9832, 0.0006, 0.9078, 0.9858, 0.0331]),
 DenseVector([13.6404, 7385.2921, 402.9602, 12.3474, 1.1507, 0.9042, 1.694, 0.7413, 0.8492, 1.3479, 0.6247, 0.4315, 0.9323, 0.7812, 1.1109, 0.4915, 2.4534, 1.4881, 0.7263, 1.4829, 0.3647, 2.7535, 0.2675, 0.7244, 1.3993, 1.5769, 0.8104, 0.9736, 1.4141, 1.2134, 1.4496, 0.941, 0.1407, 0.2418, 1.4216, 0.0222, 0.2977, 0.2807, 0.0852, 0.1059, 0.0, 0.2947, 0.0706, 0.1358, 0.323, 0.2295, 0.0, 0.0893, 0.3287, 0.0241, 0.2893, 0.3309, 0.2317]))

In [55]:
import pyspark.mllib.regression as reg

final_data_income = (
    final_data
    .map(lambda row: reg.LabeledPoint(
        row[0]
        , row[1]
        )
    )
)
final_data_income.take(2)

[LabeledPoint(0.0, [0.03067008637999638,0.14845061558793698,-0.2166562000280365,-0.035428902921321385,2.1253604076175487,-0.8191162474391422,0.6377747731414669,-2.2257823875314626,-1.0799837512449586,1.1674000007482537,0.6389894611164895,0.3251568749252254,-0.4646727543295736,0.04874821465282587,-0.5786685813762596,0.8307480441969635,-0.3164213118784984,-0.9454599015180329,1.0589898184549418,-0.4370604222171284,-0.43309553568085,-0.7278380536147465,-0.2899365550040338,-1.1360930863759908,0.9505351394321797,1.3816615961467085,0.9265109390980426,1.179623833658065,1.4008228183818396,-0.2909499987558919,-0.1282833184241006,-0.7030605487269502,-0.14361192904330972,-0.1934362479927198,0.3297437122592278,-0.022172322372396518,0.3265265819196172,0.3019401519298143,-0.08580764411145662,-0.1070613054202921,0.0,0.2600747473046429,-0.05433765529105814,-0.13840137581626044,0.16932585576117493,-0.2116905580599674,0.0,-0.09006362818252112,0.0510078567719009,-0.024162841832971647,0.3186792645007414,0.

In [57]:
mu, std = sModel.mean[3], sModel.std[3]

final_data_hours = (
    final_data
    .map(lambda row: reg.LabeledPoint(
        row[1][3] * std + mu
        , ln.Vectors.dense([row[0]] + list(row[1][0:3]) + list(row[1][4:]))
        )
    )
)
final_data_hours.take(2)

[LabeledPoint(40.0, [0.0,0.03067008637999638,0.14845061558793698,-0.2166562000280365,2.1253604076175487,-0.8191162474391422,0.6377747731414669,-2.2257823875314626,-1.0799837512449586,1.1674000007482537,0.6389894611164895,0.3251568749252254,-0.4646727543295736,0.04874821465282587,-0.5786685813762596,0.8307480441969635,-0.3164213118784984,-0.9454599015180329,1.0589898184549418,-0.4370604222171284,-0.43309553568085,-0.7278380536147465,-0.2899365550040338,-1.1360930863759908,0.9505351394321797,1.3816615961467085,0.9265109390980426,1.179623833658065,1.4008228183818396,-0.2909499987558919,-0.1282833184241006,-0.7030605487269502,-0.14361192904330972,-0.1934362479927198,0.3297437122592278,-0.022172322372396518,0.3265265819196172,0.3019401519298143,-0.08580764411145662,-0.1070613054202921,0.0,0.2600747473046429,-0.05433765529105814,-0.13840137581626044,0.16932585576117493,-0.2116905580599674,0.0,-0.09006362818252112,0.0510078567719009,-0.024162841832971647,0.3186792645007414,0.04297651932271740

In [59]:
#Splitting into training and testing:

(
    final_data_income_train
    , final_data_income_test
) = (
    final_data_income.randomSplit([0.7, 0.3])
)

In [63]:
(
    final_data_hours_train
    , final_data_hours_test
) = (
    final_data_hours.randomSplit([0.7, 0.3])
)

In [69]:
#Linear regression (benchmark):
workhours_model_lm = reg.LinearRegressionWithSGD.train(final_data_hours_train)

In [71]:
small_sample_hours = sc.parallelize(final_data_hours_test.take(10))

for t,p in zip(
    small_sample_hours
        .map(lambda row: row.label)
        .collect()
    , workhours_model_lm.predict(
        small_sample_hours
            .map(lambda row: row.features)
    ).collect()):
    print(t,p)

16.0 6.830853457539442
50.0 47.24771075208423
30.0 -1.2896479190769061
50.0 -5.99987511758531
40.0 11.62883926225445
40.0 3.232449040664951
80.0 10.2378245908929
50.0 -9.462018121270566
38.0 -5.113835478205115
60.0 23.86803758181617


In [73]:
workhours_model_lm.weights

DenseVector([54.2342, -3.0148, -2.9833, -1.8618, -0.447, -0.9443, 1.6672, 1.3634, 3.8812, -0.754, 1.2213, -2.8573, -1.9288, -2.7053, -0.8999, -1.5485, -0.6618, -3.1688, -0.1348, 2.1113, -1.7397, -0.3638, 0.6162, 1.7695, -0.2258, 2.8457, -0.1653, -0.2064, 3.855, 0.6009, -0.2912, -0.9782, -0.1446, 0.6608, 0.5371, 0.066, 0.0094, 0.0675, -0.0226, 0.1218, 0.0, 0.0754, 0.4457, 0.3121, -0.064, 0.5337, 0.0, 0.0363, -0.2317, -0.0262, -0.3481, -0.2478, -0.0437])

In [75]:
#Logistic regression:
import pyspark.mllib.classification as cl

income_model_lr = cl.LogisticRegressionWithSGD.train(final_data_income_train)

/Users/enginsan/anaconda3/lib/python3.11/site-packages/pyspark/mllib/classification.py:395: FutureWarning: Deprecated in 2.0.0. Use ml.classification.LogisticRegression or LogisticRegressionWithLBFGS.
  warnings.warn(


In [77]:
small_sample_income = sc.parallelize(final_data_income_test.take(10))

for t,p in zip(
    small_sample_income
        .map(lambda row: row.label)
        .collect()
    , income_model_lr.predict(
        small_sample_income
            .map(lambda row: row.features)
    ).collect()):
    print(t,p)

1.0 1
1.0 1
1.0 1
0.0 0
0.0 1
0.0 0
0.0 1
1.0 1
0.0 0
0.0 1


In [79]:
income_model_lr.threshold

0.5

In [81]:
income_model_lr.weights

DenseVector([0.1799, 0.7504, 0.2263, 0.2169, 0.0315, 0.0048, -0.0204, 0.0085, -0.307, 0.0893, -0.0137, 0.2225, 0.0742, 0.1643, 0.121, 0.2405, 0.1814, 0.2651, -0.109, -0.1053, 0.2143, 0.0099, -0.0604, -0.0604, 0.0592, -0.1135, -0.0554, -0.107, -0.0902, -0.0421, 0.0133, -0.1062, 0.0138, -0.0542, 0.0137, 0.0007, -0.0134, 0.0083, -0.0055, 0.0127, 0.0, -0.0019, -0.0022, -0.0078, 0.0068, -0.0042, 0.0, -0.0115, 0.0057, -0.0107, 0.0182, 0.011, 0.0051])

In [83]:
#Support Vector Machines
income_model_svm = cl.SVMWithSGD.train(
    final_data_income
#     , step=0.95
    , miniBatchFraction=1/2.0
)

24/09/17 11:06:37 WARN GradientDescent: Testing against a convergenceTol when using miniBatchFraction < 1.0 can be unstable because of the stochasticity in sampling.


In [85]:
for t,p in zip(
    small_sample_income
        .map(lambda row: row.label)
        .collect()
    , income_model_svm.predict(
        small_sample_income
            .map(lambda row: row.features)
    ).collect()):
    print(t,p)

1.0 1
1.0 1
1.0 1
0.0 0
0.0 0
0.0 0
0.0 1
1.0 1
0.0 1
0.0 1


In [87]:
income_model_svm.weights

DenseVector([0.0804, 1.0427, 0.122, 0.0509, 0.0244, 0.0036, -0.0141, 0.0248, -0.2197, -0.017, -0.1458, 0.1318, 0.1196, 0.1345, 0.1432, 0.1314, 0.2342, 0.4894, -0.109, -0.0688, 0.0823, 0.0114, -0.0348, -0.0548, 0.0098, -0.0755, 0.0157, -0.0475, -0.0793, -0.0283, 0.0199, -0.0304, 0.0082, -0.0535, 0.0052, 0.0033, -0.0072, -0.0012, -0.0095, 0.0089, 0.0, 0.0025, 0.0001, 0.0118, 0.0289, -0.0189, 0.0, 0.0049, -0.0247, -0.0131, 0.0311, 0.0147, 0.0109])

In [89]:
#Clustering Model with Kmeans:
import pyspark.mllib.clustering as clu

model_km = clu.KMeans.train(
    final_data.map(lambda row: row[1])
    , 2
    , initializationMode='random'
    , seed=666
)

In [91]:
import sklearn.metrics as m

predicted = (
    model_km
        .predict(
            final_data.map(lambda row: row[1])
        )
)
predicted = predicted.collect()

true = final_data.map(lambda row: row[0]).collect()

print(m.homogeneity_score(true, predicted))
print(m.completeness_score(true, predicted))

0.19319268237882328
0.15462507440834927


In [93]:
#Model Metrics:
import pyspark.mllib.evaluation as ev

In [95]:
true_pred_reg = (
    final_data_hours_test
    .map(lambda row: (
         float(workhours_model_lm.predict(row.features))
         , row.label))
)

metrics_lm = ev.RegressionMetrics(true_pred_reg)

/Users/enginsan/anaconda3/lib/python3.11/site-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [97]:
print('R^2: ', metrics_lm.r2)
print('Explained Variance: ', metrics_lm.explainedVariance)
print('meanAbsoluteError: ', metrics_lm.meanAbsoluteError)

R^2:  -6.944539336445464
Explained Variance:  1132.7045961809145
meanAbsoluteError:  29.632095551910552


In [99]:

true_pred_class_lr = (
    final_data_income_test
    .map(lambda row: (
        float(income_model_lr.predict(row.features))
        , row.label))
)

metrics_lr = ev.BinaryClassificationMetrics(true_pred_class_lr)

print('areaUnderPR: ', metrics_lr.areaUnderPR)
print('areaUnderROC: ', metrics_lr.areaUnderROC)

areaUnderPR:  0.4505040192340204
areaUnderROC:  0.7858291201765141


In [101]:
trainErr = (
    true_pred_class_lr
    .filter(lambda lp: lp[0] != lp[1]).count() 
    / float(true_pred_class_lr.count())
)
print("Training Error = " + str(trainErr))

Training Error = 0.27834841859046067


In [103]:
true_pred_class_svm = (
    final_data_income_test
    .map(lambda row: (
        float(income_model_svm.predict(row.features))
        , row.label))
)

metrics_svm = ev.BinaryClassificationMetrics(true_pred_class_svm)

print('areaUnderPR: ', metrics_svm.areaUnderPR)
print('areaUnderROC: ', metrics_svm.areaUnderROC)

areaUnderPR:  0.44818795160982283
areaUnderROC:  0.7795946326911545


In [105]:
trainErr = (
    true_pred_class_svm
    .filter(lambda lp: lp[0] != lp[1]).count() 
    / float(true_pred_class_svm.count())
)

print("Training Error = " + str(trainErr))

Training Error = 0.2777382284145225


In [ ]:
#Confusion matrix:

In [107]:
(
    true_pred_class_lr
    .map(lambda el: ((el), 1))
    .reduceByKey(lambda x,y: x+y)
    .take(4)
)

[((1.0, 1.0), 2153), ((0.0, 0.0), 4943), ((1.0, 0.0), 2523), ((0.0, 1.0), 214)]

In [109]:
(
    true_pred_class_svm
    .map(lambda el: ((el), 1))
    .reduceByKey(lambda x,y: x+y)
    .take(4)
)

[((1.0, 1.0), 2107), ((0.0, 0.0), 4995), ((1.0, 0.0), 2471), ((0.0, 1.0), 260)]